<a href="https://colab.research.google.com/github/herjh0405/DACON/blob/master/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9_%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON 한국토지주택공사
구내식당 식수 인원 예측 AI 경진대회

## 필요 Model Install

In [ ]:
# !pip install pycaret

# !pip install konlpy
# !pip install matplotlib -U

# 한글 폰트 사용
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import Library

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)

from pycaret.regression import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
from konlpy.tag import Kkma

import torch
import os, re
import matplotlib.pyplot as plt

plt.rcParams['font.family']=['NanumGothic', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
# %matplotlib inline

In [ ]:
# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as fm
# import os

# def change_matplotlib_font(font_download_url):
#     FONT_PATH = 'MY_FONT'
    
#     font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
#     unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
#     os.system(font_download_cmd)
#     os.system(unzip_cmd)
    
#     font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
#     for font_file in font_files:
#         fm.fontManager.addfont(font_file)

#     font_name = fm.FontProperties(fname=font_files[0]).get_name()
#     matplotlib.rc('font', family=font_name)
#     print("font family: ", plt.rcParams['font.family'])

# font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
# change_matplotlib_font(font_download_url)

## Load Data

In [ ]:
path = '/content/drive/MyDrive/DACON/Dacon_Industry_Meal/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'train.csv')

train.columns = ['일자', '요일', '정원','휴가자', '출장자', '시간외근무승인',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']
test.columns = ['일자', '요일', '정원','휴가자', '출장자', '시간외근무승인',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']

In [ ]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

## 데이터 시각화
* 시간에 따른 time_col의 변화를 관찰
    * time_col : `'정원', '휴가자', '출장자', '시간외근무승인', '재택근무자', '중식계', '석식계'`

* 삼성 SDS에서 실시한 적이 있는 대회라고 함
    * RNN에서 휴가와 날씨에 가중치를 둬서 좋은 결과를 냈다고 함 -> 참고하면 좋을 듯
    * 일기예보 데이터를 사용해보는 건 어떨까?

<br>

### 가설 1. 재택근무는 코로나 때문에 생겼을 것이다.
* 코로나 이전과 이후로, 식수 인원이 큰 차이가 있을 것이다

<br>

### 가설 2. 날씨가 큰 영향을 미칠것이다
* 월별, 계절별 데이터가 차이날 것

<br>

### 가설 3. 공휴일들과 휴가자는 큰 연관성이 있을 것이다.
* 공휴일과 그렇지 않은 날을 구분해보자.

<br>

### 궁금증 1. 조식이 과연 의미가 있을까?
* 일단 없을거라 생각되지만 확인하기 어려움


In [ ]:
time_col = list(train.columns[2:7]) + list(train.columns[-2:])

In [ ]:
plt.figure(figsize=(10, 8))
for col in time_col : 
    plt.plot(train['일자'], train[col], label=col)

plt.xlabel('날짜(일)', fontsize=10)
plt.ylabel('인원 수', fontsize=10)
plt.title('날짜 별 인원 수', fontsize=15)
plt.legend('best', labels=time_col)
plt.show()